# **Spec**

In [1]:
!nvidia-smi

Mon Sep  6 07:42:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b BM # 자기 브랜치 이름으로 변경
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 258 (delta 143), reused 153 (delta 39), pack-reused 0
Receiving objects: 100% (258/258), 2.95 MiB | 25.39 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 12.5 MB/s 
     |████████████████████████████████| 282 kB 51.3 MB/s 
     |████████████████████████████████| 2.8 MB 65.3 MB/s 
     |████████████████████████████████| 919 kB 62.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.4 MB/s 
     |████████████████████████████████| 302 kB 67.1 MB/s 
     |████████████████████████████████| 243 kB 65.2 MB/s 
     |████████████████████████████████| 119 kB 66.0 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |█████████████████████████

In [22]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


# 시드 통일하기

In [28]:
import torch
from transformers import set_seed


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# 모델 불러오기

In [29]:
# 만약 ForSequenceClassification이 붙은 모델을 사용할 경우
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [30]:
from transformers import DistilBertForSequenceClassification
from tokenization_kobert import KoBertTokenizer

model = DistilBertForSequenceClassification.from_pretrained("monologg/distilkobert", **model_config)
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512)

# model.save_pretrained("./model")
tokenizer.tokenize("안녕하세요. 오늘 날씨는 비가 계속 내리내요.")

loading configuration file https://huggingface.co/monologg/distilkobert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4d5d25a119f63fb5526032bb7cdaf522e10cccab75fe0a539eca437722feca3d.0638161d3d5ebe20c18198663a075aca59b59977a233e0ca4b40737067ea8894
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "initializer_range": 0.02,
  "label2id": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 3,
  "pad_token_id": 1,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.10.0",
  "vocab_size": 8002
}

loading weights file https://huggingface.co/monologg/distilkobert/resolve/main/pytorch_model.bin from

['▁안', '녕', '하세요', '.', '▁오늘', '▁날씨', '는', '▁비가', '▁계속', '▁내리', '내', '요', '.']

# 하이퍼 파라미터 설정

In [7]:
num_classes = 3

# 원하는 대로 고쳐서 사용
batch_size = 32
lr = 0.0001
epochs = 150

In [ ]:
# 옵티마이저와 스케줄러
# 원하는 대로 고쳐서 사용

# from torch.optim import AdamW
# from transformers import AdamW, get_linear_schedule_with_warmup


# LENGTH_OF_TRAIN_DATA = 40242
# num_training_steps = ((LENGTH_OF_TRAIN_DATA - 1) // batch_size + 1) * epochs
# optimizer = AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.1), num_training_steps)

# 만약 이 스케줄러를 사용할 경우,
# optimizer.step() 바로 다음에
# scheduler.step()을 호출해야 함

# f1 score 계산하기

In [ ]:
# 1. torchmetrics 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from torchmetrics import F1


# f1_score = F1(num_classes=num_classes)

# # 검증 단계에서 사용
# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)  # 대충 아웃풋
#     pred = torch.argmax(output, dim=1)
#     batch_f1 = f1_score(pred, labels)
#     print(batch_f1)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 2. datasets 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음
# 3번 사이킷런 사용법 쓰십시오 이건 좋지 않음

# from datasets import load_metric


# f1_score = load_metric("f1")

# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)
#     pred = torch.argmax(output, dim=1)
#     f1_score.add_batch(predictions=pred, references=labels)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 3. sklearn 사용
# from sklearn.metrics import accuracy_score, f1_score


# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "f1": f1}

# **훈련**

In [31]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

In [32]:
from data_loader import get_data_loaders


train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)
train_data = train_data.remove_columns("token_type_ids")
val_data = val_data.remove_columns("token_type_ids")
test_data = test_data.remove_columns("token_type_ids")

Using custom data configuration default-2093c11bcee5afac
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-0fa7aba7f64e24e4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-7ef3596a625ea82e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-915f026585d9b140.arrow


In [33]:
model_name = "DistilKoBERT_without_decay"
save_path = f"{path_to_drive}/{model_name}"

In [34]:
from transformers import TrainingArguments, EarlyStoppingCallback

from trainer import ImbalancedSamplerTrainer
 

# optimizer = AdamW(model.parameters(), lr =1e-4, eps = 1e-8)
# num_train_epochs = 150
# scheduler = get_polynomial_decay_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_epochs)

args = TrainingArguments(
    output_dir=save_path,
    overwrite_output_dir=True,
    learning_rate=1e-05,
    logging_dir=save_path + "/log", 
    lr_scheduler_type="linear", # (:obj:`str` or :class:`~transformers.SchedulerType`, `optional`, defaults to :obj:`"linear"`):The scheduler type to use. See the documentation of :class:`~transformers.SchedulerType` for all possible values.
    evaluation_strategy="steps",
    save_total_limit = 1,       # Only last 1 models are saved(best model). Older ones are deleted. ref= https://stackoverflow.com/questions/62525680/save-only-best-weights-with-huggingface-transformers
    load_best_model_at_end=True,  
    num_train_epochs=150,       # total number of training epochs
    seed=42,
    fp16=True,
    weight_decay=0.01,               # strength of weight decay
    metric_for_best_model="f1",
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    greater_is_better=True,
)

trainer = ImbalancedSamplerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend


In [38]:
trainer.train()

***** Running training *****
  Num examples = 40242
  Num Epochs = 150
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 377400


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.201500,0.396620,0.866503,0.670570
1000,0.198500,0.452570,0.852639,0.678171
1500,0.182700,0.466956,0.861136,0.687340
2000,0.176600,0.454417,0.861583,0.669807
2500,0.170200,0.429547,0.880590,0.702346
3000,0.170800,0.528103,0.846154,0.659012
3500,0.166400,0.496589,0.866055,0.675109
4000,0.173500,0.422500,0.877013,0.701208
4500,0.159000,0.513188,0.863372,0.672897
5000,0.140100,0.437335,0.883945,0.706900


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1314: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-7500] due to args.save_total_limit
Deleting older checkpoint [/content/drive/My

TrainOutput(global_step=23500, training_loss=0.12296386621353474, metrics={'train_runtime': 2129.6701, 'train_samples_per_second': 2834.383, 'train_steps_per_second': 177.211, 'total_flos': 2.523916204515533e+16, 'train_loss': 0.12296386621353474, 'epoch': 9.34})

In [37]:
# trainer.train(resume_from_checkpoint=True) # 끊기기전 모델 사용하여 계속 학습 진행

Loading model from /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-8000).
***** Running training *****
  Num examples = 40242
  Num Epochs = 150
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 377400
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 8000
  Will skip the first 3 epochs then the first 452 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/452 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss,Accuracy,F1
8500,0.200600,0.421809,0.863596,0.676681


***** Running Evaluation *****
  Num examples = 4472
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-8500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-8500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-8500/pytorch_model.bin


ValueError: ignored

In [36]:
trainer.save_model(output_dir=save_path)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay
Configuration saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/pytorch_model.bin


# **모델 테스트**

In [39]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.150
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [49]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tokenization_kobert import KoBertTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-16000")
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512) 

loading configuration file /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-16000/config.json
Model config DistilBertConfig {
  "_name_or_path": "monologg/distilkobert",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "initializer_range": 0.02,
  "label2id": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 3,
  "pad_token_id": 1,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.10.0",
  "vocab_size": 8002
}

loading weights file /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_without_decay/checkpoint-16000/pytorch_model.bin
A

In [50]:
# 테스트 데이터

test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/all_data_v2.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(50).tolist()

In [51]:
model.to("cpu")
model.eval()
print()

In [52]:
from data_loader import get_data_loaders


_, _, test_dataset = get_data_loaders(tokenizer, return_loader=False, use_imbalanced=False, device="cuda") # DistilKoBERT Dataset

# _, _, test_loader = get_data_loaders(tokenizer, use_imbalanced=False, device="cuda")

Using custom data configuration default-2093c11bcee5afac
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-0fa7aba7f64e24e4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-7ef3596a625ea82e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2093c11bcee5afac/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-915f026585d9b140.arrow


In [53]:
from torch.utils.data import DataLoader

test_dataset = test_dataset.remove_columns("token_type_ids")
test_loader = DataLoader(test_dataset)

In [54]:
from torchmetrics import F1
from tqdm.auto import tqdm


f1_score = F1(num_classes=3, average="macro").cuda()
# f1
model.cuda()
model.eval()

pred_array = []
labels_data = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        labels = batch.pop("labels")
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, dim=1)
        f1_score(pred, labels)
        pred_list = pred.tolist()
        pred_array.append(pred_list[0])
        labels_list = labels.tolist()
        labels_data.append(labels_list[0])

score = f1_score.compute()
print(score.item())

  0%|          | 0/5000 [00:00<?, ?it/s]

0.8813678026199341
